In [1]:
from pathlib import Path
import polars as pl
from cogmood_analysis.load import load_prolific_data, load_survey, proc_survey
import cogmood_analysis.survey_helpers as sh
import numpy as np
from joblib import Parallel, delayed
from datetime import datetime
import json
pl.Config(tbl_rows=300)
pl.Config(tbl_cols=300)

In [2]:
data_dir = Path('../data/')
survey_dir = data_dir / 'survey'
complete_dir = survey_dir / 'complete'
ongoing_dir = survey_dir / 'ongoing'
start_time = pl.Series(['2025-07-11 00:00:00']).str.to_datetime()[0]

In [3]:
bad_subids = [
    'no0z2yzyloa58hcsb5cyxxwz',
    'jvj53cg6gm44jattfxws849e',
    'fqebjziam9e7e9vnpzqghiv9',
    'lwk7rgfebcajlfttz1f3euzs',
    'b1c6cj5oy3wv9sh4qyj379s9',
    'in6dp60i65swuwbnbyjz8m6i',
    's7qczd3ccbwvvv54rkg2s3xh',
    '3y3tn37wv2libdutqxbcat3d',
    'p1h1eval1q08k2beesprnfwq'
]

In [4]:
survey_resps = [load_survey(sr_path) for sr_path in complete_dir.glob("*.json")]
srdf = proc_survey(survey_resps)


In [5]:
srdf = srdf.filter(pl.col('survey_date')>start_time).filter(~pl.col('sub_id').is_in(bad_subids))

In [6]:
group_order = ['hv', 'dep', 'anx', 'atn', 'dep_anx', 'dep_atn', 'anx_atn', 'dep_anx_atn', 'othermh']
order_mapping = {val: i for i, val in enumerate(group_order)}

In [7]:
srdf.select('age').max(), srdf.select('age').min()

(shape: (1, 1)
 ┌──────┐
 │ age  │
 │ ---  │
 │ f64  │
 ╞══════╡
 │ 84.0 │
 └──────┘,
 shape: (1, 1)
 ┌──────┐
 │ age  │
 │ ---  │
 │ f64  │
 ╞══════╡
 │ 18.0 │
 └──────┘)

In [8]:
srdf.group_by('screen_group').agg([
    pl.n_unique('sub_id').alias('n')
    ]).sort(pl.col("screen_group").replace(order_mapping)), 'foo'

(shape: (9, 2)
 ┌──────────────┬──────┐
 │ screen_group ┆ n    │
 │ ---          ┆ ---  │
 │ str          ┆ u32  │
 ╞══════════════╪══════╡
 │ hv           ┆ 1039 │
 │ dep          ┆ 153  │
 │ anx          ┆ 718  │
 │ atn          ┆ 105  │
 │ dep_anx      ┆ 943  │
 │ dep_atn      ┆ 47   │
 │ anx_atn      ┆ 231  │
 │ dep_anx_atn  ┆ 807  │
 │ othermh      ┆ 25   │
 └──────────────┴──────┘,
 'foo')

In [9]:
srdf = srdf.with_columns(
    pl.when(pl.col('sub_id') == '80mpipxnvluq0stsd3o9ln23')
    .then(pl.col('fried').str.slice(25))
    .otherwise(pl.col('fried'))
    .alias('fried')
).with_columns(
    pl.when(pl.col('sub_id') == 'svte0ghvvjo62vbd7xohqkfr')
    .then(pl.lit("Dysfunctional family situation and abuse by authority figures"))
    .otherwise(pl.col('fried'))
    .alias('fried')
).with_columns(
    pl.when(pl.col('sub_id') == 'e4wyi6h1clyi69oxty3egh91')
    .then(pl.col('fried').str.slice(0,279))
    .otherwise(pl.col('fried'))
    .alias('fried')
)

In [10]:
col_order = [
    'sub_id',
    'survey_date',
    'race__american_indian_alaska_native',
    'race__asian',
    'race__native_hawaiian_or_other_pacific_islander',
    'race__black_or_african_american',
    'race__white',
    'ethnicity__not_hispanic_or_latino',
    'ethnicity__hispanic_or_latino',
    'sex_at_birth__female',
    'sex_at_birth__male',
    'sex_at_birth__prefer_not_to_answer',
    'age',
    'ladder_resp',
    'ongoing_mentalhealth',
    'mentalhealth_daily_impact',
    'experience_depression',
    'experience_anxiety',
    'have_adhd',
    'attn__1',
    'fried',
    'mood_pro_diagnosis',
    'mood_diagnoses__mdd',
    'mood_diagnoses__pdd',
    'mood_diagnoses__premenstrual',
    'mood_diagnoses__bp1',
    'mood_diagnoses__bp2',
    'mood_diagnoses__ct',
    'mood_diagnoses__other',
    'mood_diagnoses__pna',
    'mood_first_dx_years',
    'mood_treatment__none',
    'mood_treatment__1on1',
    'mood_treatment__group',
    'mood_treatment__ssri',
    'mood_treatment__snri',
    'mood_treatment__tricylic',
    'mood_treatment__maoi',
    'mood_treatment__sari',
    'mood_treatment__tetracyclic',
    'mood_treatment__olanzapine_fluoxotine',
    'mood_treatment__buproprion',
    'mood_treatment__li',
    'mood_treatment__valproate',
    'mood_treatment__carbazepine',
    'mood_treatment__lamotrigine',
    'mood_treatment__haldol',
    'mood_treatment__atypical_antipsychotic',
    'mood_treatment__lurasidone',
    'mood_treatment__lumaterperone',
    'mood_treatment__other',
    'mood_treatment__pna',
    'mood_med_today',
    'mood_bothered',
    'mood_bothered_today',
    'anxiety_pro_diagnosis',
    'anxiety_diagnoses__gad',
    'anxiety_diagnoses__sepad',
    'anxiety_diagnoses__agoraphobia',
    'anxiety_diagnoses__specific_phobia',
    'anxiety_diagnoses__socad',
    'anxiety_diagnoses__panic_disorder',
    'anxiety_diagnoses__panic_attack',
    'anxiety_diagnoses__selective_mutism',
    'anxiety_diagnoses__other',
    'anxiety_diagnoses__pna',
    'anxiety_first_dx_years',
    'anxiety_treatment__none',
    'anxiety_treatment__1on1',
    'anxiety_treatment__group',
    'anxiety_treatment__ssri',
    'anxiety_treatment__snri',
    'anxiety_treatment__tricyclic',
    'anxiety_treatment__benzodiazepine',
    'anxiety_treatment__maoi',
    'anxiety_treatment__beta_blocker',
    'anxiety_treatment__buspirone',
    'anxiety_treatment__other',
    'anxiety_treatment__pna',
    'anxiety_med_today',
    'anxiety_bothered',
    'anxiety_bothered_today',
    'attention_pro_diagnosis',
    'attention_diagnoses__adhd',
    'attention_diagnoses__add',
    'attention_diagnoses__other',
    'attention_diagnoses__pna',
    'attention_first_dx_years',
    'attention_treatment__none',
    'attention_treatment__1on1',
    'attention_treatment__group',
    'attention_treatment__amphetamine',
    'attention_treatment__methylphenidate',
    'attention_treatment__stomoxetine',
    'attention_treatment__buproprion',
    'attention_treatment__guanfacine',
    'attention_treatment__clonidine',
    'attention_treatment__other',
    'attention_treatment__pna',
    'attention_med_today',
    'attention_bothered',
    'attention_bothered_today',
    'baars_inattentive_1',
    'baars_inattentive_2',
    'baars_inattentive_3',
    'baars_inattentive_4',
    'baars_inattentive_5',
    'baars_inattentive_6',
    'attn__2',
    'baars_inattentive_7',
    'baars_inattentive_8',
    'baars_inattentive_9',
    'baars_hyperactivity_1',
    'baars_hyperactivity_2',
    'baars_hyperactivity_3',
    'baars_hyperactivity_4',
    'baars_hyperactivity_5',
    'baars_impulsivity_1',
    'baars_impulsivity_2',
    'baars_impulsivity_3',
    'baars_impulsivity_4',
    'baars_sct_1',
    'baars_sct_2',
    'baars_sct_3',
    'baars_sct_4',
    'baars_sct_5',
    'baars_sct_6',
    'baars_sct_7',
    'baars_sct_8',
    'baars_sct_9',
    'gad7__1',
    'gad7__2',
    'gad7__3',
    'gad7__4',
    'gad7__5',
    'gad7__6',
    'gad7__7',
    'phq8__1',
    'phq8__2',
    'phq8__3',
    'phq8__4',
    'phq8__5',
    'phq8__6',
    'phq8__7',
    'phq8__8',
    'hitop_anhdep_1',
    'hitop_sepinsec_1',
    'hitop_anxwor_1',
    'hitop_welbe_1',
    'hitop_appgn_1',
    'hitop_anhdep_2',
    'hitop_sepinsec_2',
    'hitop_anxwor_2',
    'hitop_sepinsec_3',
    'attn__3',
    'hitop_socanx_1',
    'hitop_anxwor_3',
    'hitop_socanx_2',
    'hitop_hypsom_1',
    'hitop_anhdep_3',
    'hitop_cogprb_1',
    'hitop_welbe_2',
    'hitop_welbe_3',
    'hitop_appgn_2',
    'hitop_sepinsec_4',
    'hitop_socanx_3',
    'hitop_indec_1',
    'hitop_socanx_4',
    'hitop_appls_1',
    'hitop_anhdep_4',
    'hitop_appgn_3',
    'hitop_anhdep_5',
    'hitop_anhdep_6',
    'hitop_socanx_5',
    'hitop_anhdep_7',
    'hitop_insom_1',
    'hitop_panic_1',
    'hitop_indec_2',
    'hitop_welbe_4',
    'hitop_appls_2',
    'hitop_sitphb_1',
    'hitop_anxwor_4',
    'hitop_cogprb_2',
    'hitop_socanx_6',
    'hitop_sepinsec_5',
    'hitop_anhdep_8',
    'hitop_anxwor_5',
    'hitop_panic_2',
    'attn__4',
    'hitop_socanx_7',
    'hitop_socanx_8',
    'hitop_welbe_5',
    'hitop_welbe_6',
    'hitop_insom_2',
    'hitop_welbe_7',
    'hitop_sitphb_2',
    'hitop_welbe_8',
    'hitop_welbe_9',
    'hitop_cogprb_3',
    'hitop_anhdep_9',
    'hitop_welbe_10',
    'hitop_sitphb_3',
    'hitop_hypsom_2',
    'hitop_hypsom_3',
    'hitop_anxwor_6',
    'hitop_socanx_9',
    'hitop_indec_3',
    'hitop_sepinsec_6',
    'hitop_sitphb_4',
    'hitop_anxwor_7',
    'hitop_cogprb_4',
    'hitop_anhdep_10',
    'hitop_appgn_4',
    'hitop_insom_3',
    'hitop_sitphb_5',
    'hitop_shmglt_1',
    'hitop_sepinsec_7',
    'hitop_hypsom_4',
    'hitop_panic_3',
    'hitop_socanx_10',
    'hitop_panic_4',
    'hitop_hypsom_5',
    'hitop_insom_4',
    'hitop_shmglt_2',
    'hitop_panic_5',
    'hitop_shmglt_3',
    'hitop_shmglt_4',
    'attn__5',
    'hitop_appls_3',
    'hitop_sepinsec_8',
    'hitop_panic_6',
    'todaybaars_inattentive_1',
    'todaybaars_inattentive_2',
    'todaybaars_inattentive_3',
    'todaybaars_inattentive_4',
    'todaybaars_inattentive_5',
    'todaybaars_inattentive_6',
    'todaybaars_inattentive_7',
    'todaybaars_inattentive_8',
    'todaybaars_inattentive_9',
    'todaybaars_hyperactivity_1',
    'todaybaars_hyperactivity_2',
    'todaybaars_hyperactivity_3',
    'todaybaars_hyperactivity_4',
    'todaybaars_hyperactivity_5',
    'todaybaars_impulsivity_1',
    'todayattn_1',
    'todaybaars_impulsivity_2',
    'todaybaars_impulsivity_3',
    'todaybaars_impulsivity_4',
    'todaybaars_sct_1',
    'todaybaars_sct_2',
    'todaybaars_sct_3',
    'todaybaars_sct_4',
    'todaybaars_sct_5',
    'todaybaars_sct_6',
    'todaybaars_sct_7',
    'todaybaars_sct_8',
    'todaybaars_sct_9',
    'today1_none',
    'todaygad7_1',
    'todaygad7_2',
    'todaygad7_3',
    'todaygad7_4',
    'todaygad7_5',
    'todaygad7_6',
    'todaygad7_7',
    'todayphq8_1',
    'todayphq8_2',
    'todayphq8_3',
    'todayphq8_4',
    'todayphq8_5',
    'todayphq8_6',
    'todayphq8_7',
    'todayphq8_8',
    'today2_none',
    'todayhitop_anhdep_1',
    'todayhitop_sepinsec_1',
    'todayhitop_anxwor_1',
    'todayhitop_welbe_1',
    'todayhitop_appgn_1',
    'todayhitop_anhdep_2',
    'todayhitop_sepinsec_2',
    'todayhitop_anxwor_2',
    'todayhitop_sepinsec_3',
    'todayhitop_socanx_1',
    'todayhitop_anxwor_3',
    'todayhitop_socanx_2',
    'todayhitop_hypsom_1',
    'todayhitop_anhdep_3',
    'todayhitop_cogprb_1',
    'todayhitop_welbe_2',
    'todayhitop_welbe_3',
    'todayattn_2',
    'todayhitop_appgn_2',
    'todayhitop_sepinsec_4',
    'todayhitop_socanx_3',
    'todayhitop_indec_1',
    'todayhitop_socanx_4',
    'todayhitop_appls_1',
    'todayhitop_anhdep_4',
    'todayhitop_appgn_3',
    'todayhitop_anhdep_5',
    'todayhitop_anhdep_6',
    'todayhitop_socanx_5',
    'today3_none',
    'todayhitop_anhdep_7',
    'todayhitop_insom_1',
    'todayhitop_panic_1',
    'todayhitop_indec_2',
    'todayhitop_welbe_4',
    'todayhitop_appls_2',
    'todayhitop_sitphb_1',
    'todayhitop_anxwor_4',
    'todayhitop_cogprb_2',
    'todayhitop_socanx_6',
    'todayhitop_sepinsec_5',
    'todayhitop_anhdep_8',
    'todayhitop_anxwor_5',
    'todayhitop_panic_2',
    'todayhitop_socanx_7',
    'todayhitop_socanx_8',
    'todayhitop_welbe_5',
    'todayhitop_welbe_6',
    'todayhitop_insom_2',
    'todayhitop_welbe_7',
    'todayhitop_sitphb_2',
    'todayhitop_welbe_8',
    'todayhitop_welbe_9',
    'todayhitop_cogprb_3',
    'todayhitop_anhdep_9',
    'todayhitop_welbe_10',
    'todayhitop_sitphb_3',
    'todayhitop_hypsom_2',
    'todayhitop_hypsom_3',
    'today4_none',
    'todayhitop_anxwor_6',
    'todayhitop_socanx_9',
    'todayhitop_indec_3',
    'todayhitop_sepinsec_6',
    'todayhitop_sitphb_4',
    'todayhitop_anxwor_7',
    'todayhitop_cogprb_4',
    'todayhitop_anhdep_10',
    'todayhitop_appgn_4',
    'todayhitop_insom_3',
    'todayhitop_sitphb_5',
    'todayhitop_shmglt_1',
    'todayhitop_sepinsec_7',
    'todayhitop_hypsom_4',
    'todayhitop_panic_3',
    'todayhitop_socanx_10',
    'todayhitop_panic_4',
    'todayhitop_hypsom_5',
    'todayhitop_insom_4',
    'todayhitop_shmglt_2',
    'todayhitop_panic_5',
    'todayhitop_shmglt_3',
    'todayhitop_shmglt_4',
    'todayattn_3',
    'todayhitop_appls_3',
    'todayhitop_sepinsec_8',
    'todayhitop_panic_6',
    'today5_none',
    'handedness__right_handed',
    'handedness__ambidextrous',
    'handedness__left_handed',
    'fatigue',
    'meal_type__more',
    'meal_type__less',
    'meal_type__about_the_same',
    'hunger',
    'attnbin__2',
    'attnbin__3',
    'attnbin__4',
    'attnbin__5',
    'baars',
    'todaybaars',
    'phq8',
    'todayphq8',
    'gad7',
    'todaygad7',
    'hitop',
    'todayhitop',
    'baars_hyperactivity',
    'todaybaars_hyperactivity',
    'baars_impulsivity',
    'todaybaars_impulsivity',
    'baars_sct',
    'todaybaars_sct',
    'hitop_anhdep',
    'todayhitop_anhdep',
    'hitop_sepinsec',
    'todayhitop_sepinsec',
    'hitop_anxwor',
    'todayhitop_anxwor',
    'hitop_welbe',
    'todayhitop_welbe',
    'hitop_appgn',
    'todayhitop_appgn',
    'hitop_socanx',
    'todayhitop_socanx',
    'hitop_hypsom',
    'todayhitop_hypsom',
    'hitop_cogprb',
    'todayhitop_cogprb',
    'hitop_indec',
    'todayhitop_indec',
    'hitop_appls',
    'todayhitop_appls',
    'hitop_insom',
    'todayhitop_insom',
    'hitop_shmglt',
    'todayhitop_shmglt',
    'attnbin',
    'todayattn',
    'screen_group'
]

In [11]:
srdf = srdf.select(col_order)

# Add in prolific data where available

In [12]:
prolific_cols = [
    'sub_id', 
    'p_total_approvals',
    'p_computer_os',
    'p_age',
    'p_language',
    'p_student_status',
    'p_employment_status',
    'p_ses',
    'p_highest_education',
    'p_depression',
    'p_anxiety', 
    'p_attention',
    'p_mental_health_ongoing',
    'p_mental_illness_impact',
    'prolific_screen_group',
]
pdf = load_prolific_data(data_dir / 'sid_prolific_demo.tsv').select(prolific_cols)


In [13]:
psrdf = srdf.join(pdf, on='sub_id', how='left')

In [14]:
psrdf.columns

['sub_id',
 'survey_date',
 'race__american_indian_alaska_native',
 'race__asian',
 'race__native_hawaiian_or_other_pacific_islander',
 'race__black_or_african_american',
 'race__white',
 'ethnicity__not_hispanic_or_latino',
 'ethnicity__hispanic_or_latino',
 'sex_at_birth__female',
 'sex_at_birth__male',
 'sex_at_birth__prefer_not_to_answer',
 'age',
 'ladder_resp',
 'ongoing_mentalhealth',
 'mentalhealth_daily_impact',
 'experience_depression',
 'experience_anxiety',
 'have_adhd',
 'attn__1',
 'fried',
 'mood_pro_diagnosis',
 'mood_diagnoses__mdd',
 'mood_diagnoses__pdd',
 'mood_diagnoses__premenstrual',
 'mood_diagnoses__bp1',
 'mood_diagnoses__bp2',
 'mood_diagnoses__ct',
 'mood_diagnoses__other',
 'mood_diagnoses__pna',
 'mood_first_dx_years',
 'mood_treatment__none',
 'mood_treatment__1on1',
 'mood_treatment__group',
 'mood_treatment__ssri',
 'mood_treatment__snri',
 'mood_treatment__tricylic',
 'mood_treatment__maoi',
 'mood_treatment__sari',
 'mood_treatment__tetracyclic',
 'mo

In [15]:
psrdf.write_csv(survey_dir / 'survey_responses.csv')

In [16]:
psrdf.drop('survey_date').write_csv(survey_dir / 'survey_responses_public.csv')

In [17]:
data_dict = {
    'sub_id': {
        'LongName': 'Subject ID',
        'Description':'Randomly assigned subject ID'
    },
    'race__american_indian_alaska_native': {
        'LongName': 'American Indian/Alaska Native',
        'Description':'Selected American Indian/Alaska Native in response to Which most closely describes your racial group(s)?'
    },
    'race__asian': {
        'LongName': 'Asian',
        'Description':'Selected Asian in response to Which most closely describes your racial group(s)?'
    },
    'race__native_hawaiian_or_other_pacific_islander': {
        'LongName': 'Native Hawaiian or Other Pacific Islander',
        'Description':'Selected Native Hawaiian or Other Pacific Islander in response to Which most closely describes your racial group(s)?'
    },
    'race__black_or_african_american': {
        'LongName': 'Black or African American',
        'Description':'Selected Black or African American in response to Which most closely describes your racial group(s)?'
    },
    'race__white': {
        'LongName': 'White',
        'Description':'Selected White in response to Which most closely describes your racial group(s)?'
    },
    'ethnicity__not_hispanic_or_latino': {
        'LongName': 'Not Hispanic or Latino',
        'Description':'Selected Not Hispanic or Latino in response to Which most closely describes your ethnic group?'
    },
    'ethnicity__hispanic_or_latino': {
        'LongName': 'Hispanic or Latino',
        'Description':'Selected Hispanic or Latino in response to Which most closely describes your ethnic group?'
    },
    'sex_at_birth__female': {
        'LongName': 'Female',
        'Description':'Selected Female in resposne to What was your biological sex at birth?'
    },
    'sex_at_birth__male': {
        'LongName': 'Male',
        'Description':'Selected Male in response to What was your biological sex at birth?'
    },
    'sex_at_birth__prefer_not_to_answer': {
        'LongName': 'Prefer not to answer',
        'Description':'Selected Prefer no to answer in response to What was your biological sex at birth?'
    },
    'age': {
        'LongName': 'Age (years)',
        'Description':'resposne to How old are you (in years)?'
    },
    'ladder_resp': {
        'LongName': 'SES Ladder',
        'Description':'Which rung of a ladder representing socio-economic status did the participant place themselves on. 2nd rung is second highers SES. 9th rung is second lowest SES.'
    },
    'ongoing_mentalhealth': {
        'LongName': 'Ongoing mental health ',
        'Description':'Do you have – or have you had – a diagnosed, on-going mental health/illness/condition?'
    },
    'mentalhealth_daily_impact': {
        'LongName': 'Mental health daily impact',
        'Description':'Do you have any diagnosed mental health condition that is uncontrolled (by medication or intervention) and which has a significant impact on your daily life / activities?'
    },
    'experience_depression': {
        'LongName': 'Experience depression',
        'Description':'Do you experience depression?'
    },
    'experience_anxiety': {
        'LongName': 'Experience anxiety',
        'Description':'Do you experience anxiety?'
    },
    'have_adhd': {
        'LongName': 'Have ADHD',
        'Description':'Do you consider yourself to have attention deficit disorder (ADD)/attention deficit hyperactivity disorder (ADHD)?'
    },
    'attn__1': {
        'LongName': 'Willing to download',
        'Description':'Are you willing to download and run tasks on your computer? If not, the survey will complete after this page and you will be redirected to Prolific.'
    },
    'fried': {
        'LongName': 'Causes of depression',
        'Description':'In the box below, please indicate what you think is the most important cause for someone developing depression. There is no right or wrong answer, we would just like to know what you think.'
    },
    'mood_pro_diagnosis': {
        'LongName': 'Mood pro dx',
        'Description':'Have you ever been diagnosed with a mood disorder by a mental health professional? This includes: Depression, Major Depressive Disorder, Bipolar Disorder type I or II, Cyclothymic Disorder,Persistent Depressive Disorder (Dysthymia),Premenstrual Dysphoric Disorder.'
    },
    'mood_diagnoses__mdd': {
        'LongName': 'Major Depressive Disorder dx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Major Depressive Disorder in response to Which mood disorders have you been diagnosed with? Please select all that apply.'
    },
    'mood_diagnoses__pdd': {
        'LongName': 'Persistent Depressive Disorder dx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Persistent Depressive Disorder in response to Which mood disorders have you been diagnosed with? Please select all that apply.'
    },
    'mood_diagnoses__premenstrual': {
        'LongName': 'Premenstrual dx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Premenstrual Dysphoric Disorder in response to Which mood disorders have you been diagnosed with? Please select all that apply.'
    },
    'mood_diagnoses__bp1': {
        'LongName': 'Bipolar 1 dx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Bipolar I Disorder in response to Which mood disorders have you been diagnosed with? Please select all that apply.'
    },
    'mood_diagnoses__bp2': {
        'LongName': 'Bipolar 2 dx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Bipolar II Disorder in response to Which mood disorders have you been diagnosed with? Please select all that apply.'
    },
    'mood_diagnoses__ct': {
        'LongName': 'Cyclothymic dx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Cyclothymic Disorder in response to Which mood disorders have you been diagnosed with? Please select all that apply.'
    },
    'mood_diagnoses__other': {
        'LongName': 'Other mood dx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Other in response to Which mood disorders have you been diagnosed with? Please select all that apply.'
    },
    'mood_diagnoses__pna': {
        'LongName': 'Prefer not to answer mood dx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Perfer not to answer in response to Which mood disorders have you been diagnosed with? Please select all that apply.'
    },
    'mood_first_dx_years': {
        'LongName': 'Years since first mood disorder diagnosis',
        'Description':'Skipped if mood_pro_diagnosis is false. How many years ago were you first diagnosed with a mood disorder (to the best of your ability to recall)?'
    },
    'mood_treatment__none': {
        'LongName': 'No moood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected I am not receiving treatment for a mood disorder in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__1on1': {
        'LongName': 'One on one talk mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected One-on-one talk therapy with a professional in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__group': {
        'LongName': 'Group mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Group therapy in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__ssri': {
        'LongName': 'SSRI mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Selective Serotonin Reuptake Inhibitor (SSRI), such as: Prozac (fluoxotine), Celexa (citalopram), Lexapro (escitalopram), Paroxetine (Paxil), Sertraline (Zoloft) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__snri': {
        'LongName': 'SNR mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Serotonin and Norepinephrine Reuptake Inhibitor (SNRI), such as: Cymbalta (duloxetine), Effexor (venalfaxine)" in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__tricylic': {
        'LongName': 'Tricyclic mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Tricyclic Antidepressant (TCA), such as: Anafranil (clomipramine), Sinequan (doxepin) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__maoi': {
        'LongName': 'MAOI mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Monoamine Oxidase Inhibitor (MAOI), such as: Emsam (selegiline), Marplan (isocarboxaxid) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__sari': {
        'LongName': 'SARI mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Serotonin Antagonist and Reuptake Inhibitors, such as: Oleptro (trazodone), Brintellix (vortioxetine) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__tetracyclic': {
        'LongName': 'Tetracyclic mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Remeron (mirtazapine) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__olanzapine_fluoxotine': {
        'LongName': 'Olanzapine_fluoxotine mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Symbax (olanzapine/fluoxotine) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__buproprion': {
        'LongName': 'Buproprion mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Wellbutrin (buproprion) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__li': {
        'LongName': 'Li mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Lithium in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__valproate': {
        'LongName': 'Valproate mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Depakane (valproate), Epival (divalproex) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__carbazepine': {
        'LongName': 'Carbazepine mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Tegretol (carbamazepine), Trileptal (oxcarbazepine) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__lamotrigine': {
        'LongName': 'Lamotrigine mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Lamictal (lamotrigine) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__haldol': {
        'LongName': 'Haldol mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Haloperidol (haldol decanoate) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__atypical_antipsychotic': {
        'LongName': 'Atypical antipsychotic mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Abilify (aripiprazole), Saphris (asenapine), Vraylar (cariprazine), Zyprexa (olanzapine), Risperdal (risperdone) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__lurasidone': {
        'LongName': 'Lurasidone mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Latuda (lurasidone) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__lumaterperone': {
        'LongName': 'Lumaterperone mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Caplyta (lumaterperone), Seroquel (quetiapine) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__other': {
        'LongName': 'Other mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Other in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_treatment__pna': {
        'LongName': 'Prefer not to answer mood tx',
        'Description':'Skipped if mood_pro_diagnosis is false. Selected Prefer not to answer in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your mood disorder(s).'
    },
    'mood_med_today': {
        'LongName': 'Taken a mood medication today',
        'Description':'Skipped if mood_pro_diagnosis is false. Response to Have you taken medication for your mood disorder today?'
    },
    'mood_bothered': {
        'LongName': 'Bothered by mood in last two weeks',
        'Description':'Skipped if mood_pro_diagnosis is false. Response to In the past two weeks have you been bothered by symptoms of your mood disorder(s)?'
    },
    'mood_bothered_today': {
        'LongName': 'Bothered by mood today',
        'Description':'Skipped if mood_pro_diagnosis is false. Response to Have you been bothered by symptoms of your mood disorder(s) today?'
    },
    'anxiety_pro_diagnosis': {
        'LongName': 'Anxiety pro dx',
        'Description':'Response to Have you ever been diagnosed with an anxiety disorder by a mental health professional? This includes: Generalized Anxiety Disorder, Separation Anxiety Disorder, Agoraphobia, Specific Phobia, Social Anxiety Disorder (Social Phobia), Panic Disorder, Panic Attack, Selective Mutism'
    },
    'anxiety_diagnoses__gad':{
        'LongName': 'Generalized Anxiety Disorder dx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Generalized Anxiety Disorder in response to Which anxiety disorders have you been diagnosed with? Please select all that apply.'
    },
    'anxiety_diagnoses__sepad':{
        'LongName': 'Separation Anxiety Disorder dx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Separation Anxiety Disorder in response to Which anxiety disorders have you been diagnosed with? Please select all that apply.'
    },
    'anxiety_diagnoses__agoraphobia':{
        'LongName': 'Agoraphobia dx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Agoraphobia in response to Which anxiety disorders have you been diagnosed with? Please select all that apply.'
    },
    'anxiety_diagnoses__specific_phobia':{
        'LongName': 'Specific phobia dx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Specific Phobia in response to Which anxiety disorders have you been diagnosed with? Please select all that apply.'
    },
    'anxiety_diagnoses__socad':{
        'LongName': 'Social Anxiety Disorder dx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Social Anxiety Disorder in response to Which anxiety disorders have you been diagnosed with? Please select all that apply.'
    },
    'anxiety_diagnoses__panic_disorder':{
        'LongName': 'Panic Disorder dx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Panic Disorder in response to Which anxiety disorders have you been diagnosed with? Please select all that apply.'
    },
    'anxiety_diagnoses__panic_attack':{
        'LongName': 'Panic attack dx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Panic attack in response to Which anxiety disorders have you been diagnosed with? Please select all that apply.'
    },
    'anxiety_diagnoses__selective_mutism':{
        'LongName': 'Selective Mutism dx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Selective Mutism in response to Which anxiety disorders have you been diagnosed with? Please select all that apply.'
    },
    'anxiety_diagnoses__other':{
        'LongName': 'Other Anxiety dx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Other in response to Which anxiety disorders have you been diagnosed with? Please select all that apply.'
    },
    'anxiety_diagnoses__pna':{
        'LongName': 'Prefer not to answer anxieity dx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Prefer not to answer in response to Which anxiety disorders have you been diagnosed with? Please select all that apply.'
    },
    'anxiety_first_dx_years':{
        'LongName': 'Years since first anxiety disorder diagnosis',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. How many years ago were you first diagnosed with an anxiety disorder (to the best of your ability to recall)?'
    },
    'anxiety_treatment__none':{
        'LongName': 'No anxiety tx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected I am not receiving treatment for an anxiety disorder in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'anxiety_treatment__1on1':{
        'LongName': 'One on one talk anxiety tx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected One-on-one talk therapy with a professional in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'anxiety_treatment__group':{
        'LongName': 'Group anxiety tx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Group therapy in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'anxiety_treatment__ssri':{
        'LongName': 'SSRT anxiety tx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Selective Serotonin Reuptake Inhibitor (SSRI), such as: Prozac (fluozotine), Celexa (citalopram), Lexapro (escitalopram), Paroxetine (Paxil), Sertraline (Zoloft) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'anxiety_treatment__snri':{
        'LongName': 'SNRI anxiety tx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Serotonin and Norepinephrine Reuptake Inhibitor (SNRI), such as: Cymbalta (duloxetine), Effexor (venalfaxine) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'anxiety_treatment__tricyclic':{
        'LongName': 'Tricyclic anxiety tx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Tricyclic Antidepressant (TCA), such as: Anafranil (clomipramine), Sinequan (doxepin) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'anxiety_treatment__benzodiazepine':{
        'LongName': 'Benzodiazepine anxiety tx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Benzodiazepine, such as: Xanax (alprazolam), Valium (diazepam), Ativan (lorazepam), Librium (chlordiazepoxide) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'anxiety_treatment__maoi':{
        'LongName': 'MAOI anxiety tx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Monoamine Oxidase Inhibitor (MAOI), such as: Emsam (selegiline), Marplan (isocarboxaxid) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'anxiety_treatment__beta_blocker':{
        'LongName': 'Beta Blocker anxiety tx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Beta-blocker, such as: Tenormin (atenolol), Inderal (propranolol) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'anxiety_treatment__buspirone':{
        'LongName': 'Buspirone anxiety tx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected BuSpar (buspirone) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'anxiety_treatment__other':{
        'LongName': 'Other anxiety tx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Other in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'anxiety_treatment__pna':{
        'LongName': 'Prefer not to answer anxiety tx',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Selected Prefer not to answer in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'anxiety_med_today':{
        'LongName': 'Taken an anxiety medication today',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Response to Have you taken medication for your anxiety disorder today?'
    },
    'anxiety_bothered':{
        'LongName': 'Bothered by anxiety in last two weeks',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Response to In the past two weeks have you been bothered by symptoms of your anxiety disorder(s)?'
    },
    'anxiety_bothered_today':{
        'LongName': 'Bothered by anxiety today',
        'Description': 'Skipped if anxiety_pro_diagnosis is false. Response to Have you been bothered by symptoms of your anxiety disorder(s) today?'
    },    
    'attention_pro_diagnosis': {
        'LongName': 'Attention pro dx',
        'Description':'Have you ever been diagnosed with attention-deficit/hyperactivity disorder (ADHD) or attention-deficit disorder (ADD)?'
    },
    'attention_diagnoses__adhd': {
        'LongName': 'ADHD dx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected Attention-Deficit/Hyperactivity Disorder (ADHD) in response to Which attention disorders have you been diagnosed with? Please select all that apply.'
    },
    'attention_diagnoses__add': {
        'LongName': 'ADD dx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected Attention-Deficit Disorder (ADD) in response to Which attention disorders have you been diagnosed with? Please select all that apply.'
    },
    'attention_diagnoses__other': {
        'LongName': 'Other attention dx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected Other in response to Which attention disorders have you been diagnosed with? Please select all that apply.'
    },
    'attention_diagnoses__pna': {
        'LongName': 'Prefer not to answer attention dx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected Prefer not to answer in response to Which attention disorders have you been diagnosed with? Please select all that apply.'
    },
    'attention_first_dx_years': {
        'LongName': 'Years since first attention disorder diagnosis',
        'Description': 'Skipped if attention_pro_diagnosis is false. Response to How many years ago were you first diagnosed with an attention disorder (to the best of your ability to recall)?'
    },
    'attention_treatment__none': {
        'LongName': 'No attention tx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected I am not receiving treatment for an attention disorder in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'attention_treatment__1on1': {
        'LongName': 'One on one talk attention tx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected One-on-one talk therapy with a professional in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'attention_treatment__group': {
        'LongName': 'Group attention tx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected Group therapy in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'attention_treatment__amphetamine': {
        'LongName': 'Amphetamine attention tx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected Amphetamine, such as Dexedrine (dextroamphetamine), Adderall (amphetamine salts), Vyvanse (lisdexamfetamine) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'attention_treatment__methylphenidate': {
        'LongName': 'Methylphenidate attention tx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected Methylphenidate, such as Concerta, Ritalin, Focalin (dexmethylphenidate) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'attention_treatment__stomoxetine': {
        'LongName': 'Stomoxetine attention tx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected Strattera (atomoxetine) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'attention_treatment__buproprion': {
        'LongName': 'Buproprion attention tx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected Wellbutrin (buproprion) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'attention_treatment__guanfacine': {
        'LongName': 'Guanfacine attention tx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected Intuniv (guanfacine) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'attention_treatment__clonidine': {
        'LongName': 'Clonidine attention tx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected Catapres, Kapvay (clonidine) in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'attention_treatment__other': {
        'LongName': 'Other attention tx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected Other in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'attention_treatment__pna': {
        'LongName': 'Prefer not to answer attention tx',
        'Description': 'Skipped if attention_pro_diagnosis is false. Selected Prefer not to answer in response to Are you currently receiving treatment for this disorder, if so, which types? Please select all treatments that you have received in the last month that are intended to treat your anxiety disorder(s).'
    },
    'attention_med_today': {
        'LongName': 'Taken an attention medication today',
        'Description': 'Skipped if attention_pro_diagnosis is false. Response to Have you taken medication for your attention disorder today?'
    },
    'attention_bothered': {
        'LongName': 'Bothered by attention in the last two weeks',
        'Description': 'Skipped if attention_pro_diagnosis is false. Response to In the past two weeks have you been bothered by symptoms of your attention disorder(s)?'
    },
    'attention_bothered_today': {
        'LongName': 'Bothered by attentin today',
        'Description': 'Skipped if attention_pro_diagnosis is false. Response to In the past two weeks have you been bothered by symptoms of your attention disorder(s)?'
    },
    'baars_inattentive_1': {
        'LongName': 'BAARS inattentive 1',
        'Description':'Fail to give close attention to details or make careless mistakes in my work or other activities',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_inattentive_2': {
        'LongName': 'BAARS inattentive 2',
        'Description':'Difficulty sustaining my attention in tasks or fun activities',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_inattentive_3': {
        'LongName': 'BAARS inattentive 3',
        'Description':"Don't listen when spoken to directly",
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_inattentive_4': {
        'LongName': 'BAARS inattentive 4',
        'Description':"Don't follow through on instructions and fail to finish work or chores",
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_inattentive_5': {
        'LongName': 'BAARS inattentive 5',
        'Description':'Have difficulty organizing tasks and activities',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_inattentive_6': {
        'LongName': 'BAARS inattentive 6',
        'Description':'Avoid, dislike, or am reluctant to engage in tasks that require sustained mental effort',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'attn__2': {
        'LongName': 'Attention Check 2',
        'Description':'To check if you are a true respondent, please select "Very often" for this item.',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_inattentive_7': {
        'LongName': 'BAARS inattentive 7',
        'Description':'Lose things necessary for tasks or activities',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_inattentive_8': {
        'LongName': 'BAARS inattentive 8',
        'Description':'Easily distracted by extraneous stimuli or irrelevant thoughts',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_inattentive_9': {
        'LongName': 'BAARS inattentive 9',
        'Description':'Forgetful in daily activities',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_hyperactivity_1': {
        'LongName': 'BAARS hyperactivity 1',
        'Description':'Fidget with hands or feet or squirm in seat',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_hyperactivity_2': {
        'LongName': 'BAARS hyperactivity 2',
        'Description':'Leave my seat in classrooms or in other situations in which remaining seated is expected',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_hyperactivity_3': {
        'LongName': 'BAARS hyperactivity 3',
        'Description':'Shift around excessively or feel restless or hemmed in',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_hyperactivity_4': {
        'LongName': 'BAARS hyperactivity 4',
        'Description':'Have difficulty engaging in leisure activities quietly (feel uncomfortable, or am loud or noisy',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_hyperactivity_5': {
        'LongName': 'BAARS hyperactivity 5',
        'Description':'I am "on the go" or act as if "driven by a motor" (or I feel like I have to be busy or always doing something)',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_impulsivity_1': {
        'LongName': 'BAARS Impulsivity 1',
        'Description':'Talk excessively (in social situations)',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_impulsivity_2': {
        'LongName': 'BAARS Impulsivity 2',
        'Description':"Blurt out answers before questions have been completed, complete others' sentences, or jump the gun",
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_impulsivity_3': {
        'LongName': 'BAARS Impulsivity 3',
        'Description':'Have difficulty awaiting my turn',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_impulsivity_4': {
        'LongName': 'BAARS Impulsivity 4',
        'Description':'Interrupt or intrude on others (butt into conversations or activities without permission or take over what others are doing)BAARS Sluggish Cognitive Tempo',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_sct_1': {
        'LongName': 'BAARS Sluggish Cognitive Tempo 1',
        'Description':'Prone to daydreaming when I should be concentrating on something or working',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_sct_2': {
        'LongName': 'BAARS Sluggish Cognitive Tempo 2',
        'Description':'Have trouble staying alert or awake in boring situations',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_sct_3': {
        'LongName': 'BAARS Sluggish Cognitive Tempo 3',
        'Description':'Easily confused',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_sct_4': {
        'LongName': 'BAARS Sluggish Cognitive Tempo 4',
        'Description':'Easily Bored',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_sct_5': {
        'LongName': 'BAARS Sluggish Cognitive Tempo 5',
        'Description':'Spacey or "in a fog',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_sct_6': {
        'LongName': 'BAARS Sluggish Cognitive Tempo 6',
        'Description':'Lethargic, more tired than others',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_sct_7': {
        'LongName': 'BAARS Sluggish Cognitive Tempo 7',
        'Description':'Underactive or have less energy than others',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_sct_8': {
        'LongName': 'BAARS Sluggish Cognitive Tempo 8',
        'Description':'Slow moving',
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'baars_sct_9': {
        'LongName': 'BAARS Sluggish Cognitive Tempo 9',
        'Description':"I don't seem to process information as quickly or as accurately as others",
        'Stem': 'For each item, please select the option that describes your behavior during the ****last 2 weeks****.',
        'Options': {'Never or rarely': 1, 'Sometimes': 2, 'Often': 3, 'Very Often': 4}
    },
    'gad7__1': {
        'LongName': 'GAD-7 1',
        'Description':'Feeling nervous, anxious or on edge',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'gad7__2': {
        'LongName': 'GAD-7 2',
        'Description':'Not being able to stop or control worrying',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'gad7__3': {
        'LongName': 'GAD-7 3',
        'Description':'Worrying too much about different things',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'gad7__4': {
        'LongName': 'GAD-7 4',
        'Description':'Trouble relaxing',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'gad7__5': {
        'LongName': 'GAD-7 5',
        'Description':'Being so restless that it is hard to sit still',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'gad7__6': {
        'LongName': 'GAD-7 6',
        'Description':'Becoming easily annoyed or irritable',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'gad7__7': {
        'LongName': 'GAD-7 7',
        'Description':'Feeling afraid, as if something awful might happen',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'phq8__1': {
        'LongName': 'PHQ-8 1',
        'Description':'Little interest or pleasure in doing things',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'phq8__2': {
        'LongName': 'PHQ-8 2',
        'Description':'Feeling down, depressed, irritable or hopeless',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'phq8__3': {
        'LongName': 'PHQ-8 3',
        'Description':'Trouble falling or staying asleep, or sleeping too much',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'phq8__4': {
        'LongName': 'PHQ-8 4',
        'Description':'Feeling tired or having little energy',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'phq8__5': {
        'LongName': 'PHQ-8 5',
        'Description':'Poor appetite or overeating',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'phq8__6': {
        'LongName': 'PHQ-8 6',
        'Description':'Feeling bad about yourself – or that you are a failure or have let yourself or your family down',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'phq8__7': {
        'LongName': 'PHQ-8 7',
        'Description':'Trouble concentrating on things, such as school work, reading or watching television',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'phq8__8': {
        'LongName': 'PHQ-8 8',
        'Description':'Moving or speaking so slowly that other people could have noticed? Or the opposite – being so fidgety or restless that you have been moving around a lot more than usual',
        'Stem': 'Over the **last two weeks**, how often have you been bothered by the following problems?',
        'Options': {'Not at all': 0, 'Several Days': 1, 'More than half the days': 2, 'Nearly every day': 3}
    },
    'hitop_anhdep_1': {
        'LongName': 'HiTOP Anhedonic Depression 1',
        'Description':'I had very little energy.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_sepinsec_1': {
        'LongName': 'HiTOP Seperation Insecurity 1',
        'Description':'I worried that others would abandon me.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anxwor_1': {
        'LongName': 'HiTOP Anxious Worry 1',
        'Description':'Thoughts were racing through my head.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_welbe_1': {
        'LongName': 'HiTOP Wellbeing 1',
        'Description':'It was easy for me to laugh.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_appgn_1': {
        'LongName': 'HiTOP Appetite Gain 1',
        'Description':'I stuffed myself with food.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anhdep_2': {
        'LongName': 'HiTOP Anhedonic Depression 2',
        'Description':'I was unable to enjoy things like I normally do.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_sepinsec_2': {
        'LongName': 'HiTOP Seperation Insecurity 2',
        'Description':'I wanted other people to take care of me.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anxwor_2': {
        'LongName': 'HiTOP Anxious Worry 2',
        'Description':'I had a lot of nervous energy.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_sepinsec_3': {
        'LongName': 'HiTOP Seperation Insecurity 3',
        'Description':'I wanted someone else to make decisions for me.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'attn__3': {
        'LongName': 'Attention check 3',
        'Description':'To check if you are a true respondent, please select "Moderately" for this item.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_socanx_1': {
        'LongName': 'HiTOP Social Anxiety 1',
        'Description':'I avoided performing or giving a talk in front of others.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anxwor_3': {
        'LongName': 'HiTOP Anxious Worry 3',
        'Description':'I was overwhelmed by anxiety.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_socanx_2': {
        'LongName': 'HiTOP Social Anxiety 2',
        'Description':'I avoided situations in which others were likely to watch me.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_hypsom_1': {
        'LongName': 'HiTOP Hyposomnia 1',
        'Description':'I felt like I could go for days without sleeping.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anhdep_3': {
        'LongName': 'HiTOP Anhedonic Depression 3',
        'Description':'Nothing seemed interesting to me.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_cogprb_1': {
        'LongName': 'HiTOP Cognitive Problems 1',
        'Description':'I could not stay focused on what I was doing.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_welbe_2': {
        'LongName': 'HiTOP Wellbeing 2',
        'Description':'I looked forward to things with enjoyment.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_welbe_3': {
        'LongName': 'HiTOP Wellbeing 3',
        'Description':'I felt that I had a lot to look forward to.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_appgn_2': {
        'LongName': 'HiTOP Appetite Gain 2',
        'Description':'I thought a lot about food.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_sepinsec_4': {
        'LongName': 'HiTOP Seperation Insecurity 4',
        'Description':'I felt insecure about important relationships in my life.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_socanx_3': {
        'LongName': 'HiTOP Social Anxiety 3',
        'Description':'I felt uncomfortable being the center of attention.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_indec_1': {
        'LongName': 'HiTOP Indecisiveness 1',
        'Description':'I was indecisive.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_socanx_4': {
        'LongName': 'HiTOP Social Anxiety 4',
        'Description':'I felt self-conscious around others.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_appls_1': {
        'LongName': 'HiTOP Appetite Loss 1',
        'Description':'My appetite was poor.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anhdep_4': {
        'LongName': 'HiTOP Anhedonic Depression 4',
        'Description':'I felt depressed.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_appgn_3': {
        'LongName': 'HiTOP Appetite Gain ',
        'Description':'I could not keep myself from eating.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anhdep_5': {
        'LongName': 'HiTOP Anhedonic Depression 5',
        'Description':"I didn’t look forward to seeing friends or family.",
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anhdep_6': {
        'LongName': 'HiTOP Anhedonic Depression 6',
        'Description':'It took a lot of effort to do everyday activities.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_socanx_5': {
        'LongName': 'HiTOP Social Anxiety 5',
        'Description':'I found it difficult to speak up in front of others.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anhdep_7': {
        'LongName': 'HiTOP Anhedonic Depression 7',
        'Description':"It felt like there wasn't anything interesting or fun to do.",
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_insom_1': {
        'LongName': 'HiTOP Insomnia 1',
        'Description':'I slept very poorly.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_panic_1': {
        'LongName': 'HiTOP Panic 1',
        'Description':'My hands were cold or sweaty.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_indec_2': {
        'LongName': 'HiTOP Indecisiveness 2',
        'Description':'I had trouble making up my mind.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_welbe_4': {
        'LongName': 'HiTOP Wellbeing 4',
        'Description':'I was proud of myself.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_appls_2': {
        'LongName': 'HiTOP Appetite Loss 2',
        'Description':'I lost a significant amount of weight without even trying.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_sitphb_1': {
        'LongName': 'HiTOP Situational Phobia 1',
        'Description':'I avoided riding in elevators.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anxwor_4': {
        'LongName': 'HiTOP Anxious Worry 4',
        'Description':'I felt tense.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_cogprb_2': {
        'LongName': 'HiTOP Cognitive Problems 2',
        'Description':'I was easily distracted.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_socanx_6': {
        'LongName': 'HiTOP Social Anxiety 6',
        'Description':'I felt shy around other people.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_sepinsec_5': {
        'LongName': 'HiTOP Seperation Insecurity 5',
        'Description':'I often felt jealous.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anhdep_8': {
        'LongName': 'HiTOP Anhedonic Depression 8',
        'Description':'I was a lot less talkative than usual.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anxwor_5': {
        'LongName': 'HiTOP Anxious Worry 5',
        'Description':'I worried about almost everything.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_panic_2': {
        'LongName': 'HiTOP Panic 2',
        'Description':'I felt dizzy or lightheaded.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'attn__4': {
        'LongName': 'Attention check 4',
        'Description':'I felt like selecting a lot.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_socanx_7': {
        'LongName': 'HiTOP Social Anxiety 7',
        'Description':'I had difficulty making eye contact with others',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_socanx_8': {
        'LongName': 'HiTOP Social Anxiety 8',
        'Description':'I felt socially awkward.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_welbe_5': {
        'LongName': 'HiTOP Wellbeing 5',
        'Description':'I felt like I had a lot of interesting things to do.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_welbe_6': {
        'LongName': 'HiTOP Wellbeing 6',
        'Description':'I found compliments very encouraging.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_insom_2': {
        'LongName': 'HiTOP Insomnia 2',
        'Description':'I had trouble staying asleep.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_welbe_7': {
        'LongName': 'HiTOP Wellbeing 7',
        'Description':'I felt cheerful.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_sitphb_2': {
        'LongName': 'HiTOP Situational Phobia 2',
        'Description':'I was afraid of flying.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_welbe_8': {
        'LongName': 'HiTOP Wellbeing 8',
        'Description':'I felt optimistic.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_welbe_9': {
        'LongName': 'HiTOP Wellbeing 9',
        'Description':'I felt like I was having a lot of fun.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_cogprb_3': {
        'LongName': 'HiTOP Cognitive Problems 3',
        'Description':'I had trouble remembering things.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anhdep_9': {
        'LongName': 'HiTOP Anhedonic Depression 9',
        'Description':'I felt emotionally numb.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_welbe_10': {
        'LongName': 'HiTOP Wellbeing 10',
        'Description':'I felt good about myself.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_sitphb_3': {
        'LongName': 'HiTOP Situational Phobia 3',
        'Description':'I was afraid of heights.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_hypsom_2': {
        'LongName': 'HiTOP Hyposomnia 2',
        'Description':'I needed much less sleep than usual.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_hypsom_3': {
        'LongName': 'HiTOP Hyposomnia 3',
        'Description':'I did not feel tired, even though I was sleeping less than usual.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anxwor_6': {
        'LongName': 'HiTOP Anxious Worry 6',
        'Description':'I felt nervous and "on edge."',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_socanx_9': {
        'LongName': 'HiTOP Social Anxiety 9',
        'Description':'I was uncomfortable entering a room when others already were present.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_indec_3': {
        'LongName': 'HiTOP Indecisiveness 3',
        'Description':'It was difficult for me to make decisions.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_sepinsec_6': {
        'LongName': 'HiTOP Seperation Insecurity 6',
        'Description':'I could not handle rejection.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_sitphb_4': {
        'LongName': 'HiTOP Situational Phobia 4',
        'Description':'I became very anxious during a storm.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anxwor_7': {
        'LongName': 'HiTOP Anxious Worry 7',
        'Description':'I felt very stressed.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_cogprb_4': {
        'LongName': 'HiTOP Cognitive Problems 4',
        'Description':'I was unable to keep my mind on what I was doing.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_anhdep_10': {
        'LongName': 'HiTOP Anhedonic Depression 10',
        'Description':'Nothing made me laugh.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_appgn_4': {
        'LongName': 'HiTOP Appetite Gain ',
        'Description':'I ate even when I was not really hungry.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_insom_3': {
        'LongName': 'HiTOP Insomnia 3',
        'Description':'I lay awake for a long time before falling asleep.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_sitphb_5': {
        'LongName': 'HiTOP Situational Phobia 5',
        'Description':'I was afraid of the dark.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_shmglt_1': {
        'LongName': 'HiTOP Shame/Guilt 1',
        'Description':'I felt guilty.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_sepinsec_7': {
        'LongName': 'HiTOP Seperation Insecurity 7',
        'Description':'I felt that I needed the approval of others.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_hypsom_4': {
        'LongName': 'HiTOP Hyposomnia 4',
        'Description':'I felt like I could keep going and going without ever getting tired.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_panic_3': {
        'LongName': 'HiTOP Panic 3',
        'Description':'I was trembling or shaking.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_socanx_10': {
        'LongName': 'HiTOP Social Anxiety 10',
        'Description':'I was uncomfortable meeting new people.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_panic_4': {
        'LongName': 'HiTOP Panic 4',
        'Description':'My heart was racing or pounding.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_hypsom_5': {
        'LongName': 'HiTOP Hyposomnia 5',
        'Description':'I had days when I never got tired.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_insom_4': {
        'LongName': 'HiTOP Insomnia 4',
        'Description':'I woke up early and could not get back to sleep.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_shmglt_2': {
        'LongName': 'HiTOP Shame/Guilt 2',
        'Description':'I felt ashamed of things I had done.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_panic_5': {
        'LongName': 'HiTOP Panic 5',
        'Description':'I was short of breath.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_shmglt_3': {
        'LongName': 'HiTOP Shame/Guilt 3',
        'Description':'I was disgusted with myself.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_shmglt_4': {
        'LongName': 'HiTOP Shame/Guilt 4',
        'Description':'I blamed myself for things.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'attn__5': {
        'LongName': 'Attention check 5',
        'Description':'I was this is a check question select not at all.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_appls_3': {
        'LongName': 'HiTOP Appetite Loss 3',
        'Description':'I did not feel much like eating.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_sepinsec_8': {
        'LongName': 'HiTOP Seperation Insecurity 8',
        'Description':'I could not stand being alone.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'hitop_panic_6': {
        'LongName': 'HiTOP Panic 6',
        'Description':'I felt nauseated.',
        'Stem': 'Have there been significant times during the **last two weeks** during which the following statements applied to you?',
        'Options': {'Not at all': 0, 'A little': 1, 'Moderately': 2, 'A lot': 3}
    },
    'todaybaars_inattentive_1': {
        'LongName': '',
        'Description':'',
    },
    'todaybaars_inattentive_2': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_inattentive_3': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_inattentive_4': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_inattentive_5': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_inattentive_6': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_inattentive_7': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_inattentive_8': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_inattentive_9': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_hyperactivity_1': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_hyperactivity_2': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_hyperactivity_3': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_hyperactivity_4': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_hyperactivity_5': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_impulsivity_1': {
        'LongName': '',
        'Description':''
    },
    'todayattn_1': {
        'LongName': 'Symptoms Today Attention check 1',
        'Description': 'Please select this item.',
        'Stem': 'Please select the statements that apply to you **today**'
    },
    'todaybaars_impulsivity_2': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_impulsivity_3': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_impulsivity_4': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_sct_1': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_sct_2': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_sct_3': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_sct_4': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_sct_5': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_sct_6': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_sct_7': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_sct_8': {
        'LongName': '',
        'Description':''
    },
    'todaybaars_sct_9': {
        'LongName': '',
        'Description':''
    },
    'today1_none': {
        'LongName': 'Today1 none',
        'Description': 'None on page 1 of symptoms today checklist'
    },
    'todaygad7_1': {
        'LongName': '',
        'Description':''
    },
    'todaygad7_2': {
        'LongName': '',
        'Description':''
    },
    'todaygad7_3': {
        'LongName': '',
        'Description':''
    },
    'todaygad7_4': {
        'LongName': '',
        'Description':''
    },
    'todaygad7_5': {
        'LongName': '',
        'Description':''
    },
    'todaygad7_6': {
        'LongName': '',
        'Description':''
    },
    'todaygad7_7': {
        'LongName': '',
        'Description':''
    },
    'todayphq8_1': {
        'LongName': '',
        'Description':''
    },
    'todayphq8_2': {
        'LongName': '',
        'Description':''
    },
    'todayphq8_3': {
        'LongName': '',
        'Description':''
    },
    'todayphq8_4': {
        'LongName': '',
        'Description':''
    },
    'todayphq8_5': {
        'LongName': '',
        'Description':''
    },
    'todayphq8_6': {
        'LongName': '',
        'Description':''
    },
    'todayphq8_7': {
        'LongName': '',
        'Description':''
    },
    'todayphq8_8': {
        'LongName': '',
        'Description':''
    },
    'today2_none': {
        'LongName': 'Today2 none',
        'Description': 'None on page 2 of symptoms today checklist'
    },
    'todayhitop_anhdep_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_sepinsec_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anxwor_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_welbe_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_appgn_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anhdep_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_sepinsec_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anxwor_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_sepinsec_3': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_socanx_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anxwor_3': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_socanx_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_hypsom_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anhdep_3': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_cogprb_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_welbe_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_welbe_3': {
        'LongName': '',
        'Description':''
    },
    'todayattn_2': {
        'LongName': 'Symptoms Today Attention check 2',
        'Description': 'Please select this item.',
        'Stem': 'Please select the statements that apply to you **today**'
    },
    'todayhitop_appgn_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_sepinsec_4': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_socanx_3': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_indec_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_socanx_4': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_appls_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anhdep_4': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_appgn_3': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anhdep_5': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anhdep_6': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_socanx_5': {
        'LongName': '',
        'Description':''
    },
    'today3_none': {
        'LongName': 'Today3 none',
        'Description': 'None on page 3 of symptoms today checklist'
    },
    'todayhitop_anhdep_7': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_insom_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_panic_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_indec_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_welbe_4': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_appls_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_sitphb_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anxwor_4': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_cogprb_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_socanx_6': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_sepinsec_5': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anhdep_8': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anxwor_5': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_panic_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_socanx_7': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_socanx_8': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_welbe_5': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_welbe_6': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_insom_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_welbe_7': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_sitphb_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_welbe_8': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_welbe_9': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_cogprb_3': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anhdep_9': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_welbe_10': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_sitphb_3': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_hypsom_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_hypsom_3': {
        'LongName': '',
        'Description':''
    },
    'today4_none': {
        'LongName': 'Today4 none',
        'Description': 'None on page 4 of symptoms today checklist'
    },
    'todayhitop_anxwor_6': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_socanx_9': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_indec_3': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_sepinsec_6': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_sitphb_4': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anxwor_7': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_cogprb_4': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_anhdep_10': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_appgn_4': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_insom_3': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_sitphb_5': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_shmglt_1': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_sepinsec_7': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_hypsom_4': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_panic_3': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_socanx_10': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_panic_4': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_hypsom_5': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_insom_4': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_shmglt_2': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_panic_5': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_shmglt_3': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_shmglt_4': {
        'LongName': '',
        'Description':''
    },
    'todayattn_3': {
        'LongName': 'Symptoms Today Attention check 3',
        'Description': 'Please select this item.',
        'Stem': 'Please select the statements that apply to you **today**'
    },
    'todayhitop_appls_3': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_sepinsec_8': {
        'LongName': '',
        'Description':''
    },
    'todayhitop_panic_6': {
        'LongName': '',
        'Description':''
    },
    'today5_none': {
        'LongName': 'Today5 none',
        'Description': 'None on page 5 of symptoms today checklist'
    },
    'handedness__right_handed': {
        'LongName': 'Right handed',
        'Description':'Are you right handed or left handed?'
    },
    'handedness__ambidextrous': {
        'LongName': 'Ambidextrous',
        'Description':'Are you right handed or left handed?'
    },
    'handedness__left_handed': {
        'LongName': 'Left handed',
        'Description':'Are you right handed or left handed?'
    },
    'fatigue': {
        'LongName': 'Fatique',
        'Description':'How tired do you feel right now? 1 is not tired, 9 is very tired'
    },
    'meal_type__more': {
        'LongName': 'Eaten more',
        'Description':'By this time today, have you eaten more, less, or about the same as you usually have by this time?'
    },
    'meal_type__less': {
        'LongName': 'Eaten less',
        'Description':'By this time today, have you eaten more, less, or about the same as you usually have by this time?'
    },
    'meal_type__about_the_same': {
        'LongName': 'Eaten about the same',
        'Description':'By this time today, have you eaten more, less, or about the same as you usually have by this time?'
    },
    'hunger': {
        'LongName': 'Hunger right now',
        'Description':'How hungry do you feel right now? 1 is not hungry, 9 is very hungry.'
    },
    'attnbin__2': {
        'LongName': 'Atention Check 2 correct',
        'Description': 'Did the participant respond to attention check 2 correctly'
    },
    'attnbin__3': {
        'LongName': 'Atention Check 3 correct',
        'Description': 'Did the participant respond to attention check 3 correctly'
    },
    'attnbin__4': {
        'LongName': 'Atention Check 4 correct',
        'Description': 'Did the participant respond to attention check 4 correctly'
    },
    'attnbin__5': {
        'LongName': 'Atention Check 5 correct',
        'Description': 'Did the participant respond to attention check 5 correctly'
    },
    'baars': {
        'LongName': 'BAARS total score',
        'Description':'Sum of items on the BAARS'
    },
    'todaybaars': {
        'LongName': 'BAARS symptoms today',
        'Description':'Number of symptoms from the BAARS experienced today'
    },
    'phq8': {
        'LongName': 'PHQ-8 total score',
        'Description':'Sum of items on the PHQ-8'
    },
    'todayphq8': {
        'LongName': 'PHQ-8 symptoms today',
        'Description': 'Number of symptoms from the PHQ-8 experienced today'
    },
    'gad7': {
        'LongName': 'GAD-7 total score',
        'Description': 'Sum of items on the GAD-7'
    },
    'todaygad7': {
        'LongName': 'GAD-7 symptoms today',
        'Description': 'Number of symptoms from the GAD-7 experienced today'
    },
    'hitop': {
        'LongName': 'HiTOP total score',
        'Description': 'Sum of HiTOP items'
    },
    'todayhitop': {
        'LongName': 'HiTOP symptoms today',
        'Description': 'Number of symptoms from the HiTOP scales experienced today'
    },
    'baars_hyperactivity': {
        'LongName': 'BAARS Hyperactivity sum',
        'Description': 'Sum of items on the BAARS Hyperactivity subscale'
    },
    'todaybaars_hyperactivity': {
        'LongName': 'BAARS Hyperactivity symptoms today',
        'Description': 'Number of symptoms from the BAARS Hyperactivity scales experienced today'
    },
    'baars_impulsivity': {
        'LongName': 'BAARS Impulsivity sum',
        'Description': 'Sum of items on the BAARS Impulsivity subscale'
    },
    'todaybaars_impulsivity': {
        'LongName': 'BAARS Impulsivity symptoms today',
        'Description': 'Number of symptoms from the BAARS Impulsivity subscale experienced today'
    },
    'baars_sct': {
        'LongName': 'BAARS Sluggish Cognitive Tempo sum',
        'Description':  'Sum of items on the BAARS Sluggish Cognitive Tempo subscale'
    },
    'todaybaars_sct': {
        'LongName': 'BAARS Sluggish Cognitive Tempo symptoms today',
        'Description': 'Number of symptoms from the BAARS Sluggish Cognitive Tempo subscale experienced today'
    },
    'hitop_anhdep': {
        'LongName': 'HiTOP Anhedonic Depression sum',
        'Description':  'Sum of items on the HiTOP Anhedonic Depression subscale'
    },
    'todayhitop_anhdep': {
        'LongName': 'HiTOP Anhedonic Depression symptoms today',
        'Description': 'Number of symptoms from the HiTOP Anhedonic Depression subscale experienced today'
    },
    'hitop_sepinsec': {
        'LongName': 'HiTOP Seperation Insecurity sum',
        'Description':  'Sum of items on the HiTOP Seperation Insecurity subscale'
    },
    'todayhitop_sepinsec': {
        'LongName': 'HiTOP Seperation Insecurity symptoms today',
        'Description': 'Number of symptoms from the HiTOP Seperation Insecurity subscale experienced today'
    },
    'hitop_anxwor': {
        'LongName': 'HiTOP Anxious Worry sum',
        'Description':  'Sum of items on the HiTOP Anxious Worry subscale'
    },
    'todayhitop_anxwor': {
        'LongName': 'HiTOP Anxious Worry symptoms today',
        'Description': 'Number of symptoms from the HiTOP Anxious Worry subscale experienced today'
    },
    'hitop_welbe': {
        'LongName': 'HiTOP Wellbeing sum',
        'Description':  'Sum of items on the HiTOP Wellbeing subscale'
    },
    'todayhitop_welbe': {
        'LongName': 'HiTOP Wellbeing items today',
        'Description': 'Number of items from the HiTOP Wellbeing subscale experienced today'
    },
    'hitop_appgn': {
        'LongName': 'HiTOP Appetite Gain sum',
        'Description':  'Sum of items on the HiTOP Appetite Gain subscale'
    },
    'todayhitop_appgn': {
        'LongName': 'HiTOP Appetite Gain symptoms today',
        'Description': 'Number of symptoms from the HiTOP Appetite Gain subscale experienced today'
    },
    'hitop_socanx': {
        'LongName': 'HiTOP Social Anxiety sum',
        'Description':  'Sum of items on the HiTOP Social Anxiety subscale'
    },
    'todayhitop_socanx': {
        'LongName': 'HiTOP Social Anxiety symptoms today',
        'Description': 'Number of symptoms from the HiTOP Social Anxiety subscale experienced today'
    },
    'hitop_hypsom': {
        'LongName': 'HiTOP Hyposomnia sum',
        'Description':  'Sum of items on the HiTOP Hyposomnia subscale'
    },
    'todayhitop_hypsom': {
        'LongName':  'HiTOP Hyposomnia symptoms today',
        'Description': 'Number of symptoms from the HiTOP Hyposomnia subscale experienced today'
    },
    'hitop_cogprb': {
        'LongName': 'HiTOP Cognitive Problems sum',
        'Description':  'Sum of items on the HiTOP Cognitive Problems subscale'
    },
    'todayhitop_cogprb': {
        'LongName': 'HiTOP Cognitive Problems',
        'Description': 'Number of symptoms from the HiTOP Cognitive Problems subscale experienced today'
    },
    'hitop_indec': {
        'LongName': 'HiTOP Indecisiveness sum',
        'Description':  'Sum of items on the HiTOP Indecisiveness subscale'
    },
    'todayhitop_indec': {
        'LongName': 'HiTOP Indecisiveness symptoms today',
        'Description': 'Number of symptoms from the HiTOP Indecisiveness subscale experienced today'
    },
    'hitop_appls': {
        'LongName': 'HiTOP Appetite Loss sum',
        'Description':  'Sum of items on the HiTOP Appetite Loss subscale'
    },
    'todayhitop_appls': {
        'LongName': 'HiTOP Appetite Loss symptoms today',
        'Description': 'Number of symptoms from the HiTOP Appetite Loss subscale experienced today'
    },
    'hitop_insom': {
        'LongName': 'HiTOP Insomnia sum',
        'Description':  'Sum of items on the HiTOP Insomnia subscale'
    },
    'todayhitop_insom': {
        'LongName': 'HiTOP Insomnia symptoms today',
        'Description': 'Number of symptoms from the HiTOP Insomnia subscale experienced today'
    },
    'hitop_shmglt': {
        'LongName': 'HiTOP Shame/Guilt sum',
        'Description':  'Sum of items on the HiTOP Shame/Guilt subscale'
    },
    'todayhitop_shmglt': {
        'LongName': 'HiTOP Shame/Guilt symptoms today',
        'Description': 'Number of symptoms from the HiTOP Shame/Guilt subscale experienced today'
    },
    'attnbin': {
        'LongName': 'Attention check score',
        'Description': 'Number of attention check items correctly answered in the survey. Must have been 3 or higher to be included in this data.'
    },
    'todayattn': {
        'LongName': 'Today checklist attention checks correct',
        'Description': 'Number of attetion items in the symptoms today checklist correctly answered'
    },
    'screen_group': {
        'LongName': 'Screening Group',
        'Description': 'Which screening group would the participant have been in if they had been assigned to screening groups based on their respones to the CogMood Survey. Calculated from experience_depression, experience_anxiety, have_adhd, and ongoing_mentalhealth.'
    },
    'p_total_approvals':{
        'LongName': 'Prolific total approvals',
        'Description': 'Number of approved jobs on Prolific'
    },
    'p_computer_os':{
        'LongName': 'Prolific Computer OS',
        'Description': "Computer OS as reported in Prolific's screening question: Which operating system does your primary personal computer run?",
        'Options': ['Chrome OS', 'macOS', 'Ubuntu', 'Windows 10 Home Edition', 'Windows 10 Pro Edition', 'Windows 10 (Other Editions)', 'Windows 10 (Unsure of the edition)', 'Windows (Other versions)']
    },
    'p_age':{
        'LongName': 'Prolific Age',
        'Description': 'Age as reported by Prolific'
    },
    'p_language':{
        'LongName': 'Prolific First Language',
        'Description': "First Language as reported by Prolific's screening question: What is your first language?"
    },
    'p_student_status': {
        'LongName': 'Prolific Student Status',
        'Description': "Student Status as reported in Prolific's screening question: Are you a student?",
        'Options': ['Yes', 'No']
    },
    'p_employment_status': {
        'LongName': 'Prolific Employment Status',
        'Description': "Employment Status as reported in Prolific's screening question: What is your employment status?",
        'Options': ['Full-Time', 'Part-Time', 'Due to start a new job within the next month', 'Unemployed (and job seeking)', "Not in paid work (e.g. homemaker', 'retired or disabled)", 'Other']
    },
    'p_ses':{
        'LongName': 'Prolific Socioeconomic Status',
        'Description': "Socioeconmic status as reported in Prolific's screening question: Where would you put yourself on the socioeconomic ladder?",
    },
    'p_highest_education': {
        'LongName': 'Prolific Highest education level completed',
        'Description': "Highest education level completed as reported in Prolific's screening question: Which of these is the highest level of education you have completed?",
        'Options': [
            'No formal qualifications',
            'Secondary education (e.g. GED/GCSE)',
            'High school diploma/A-levels',
            'Technical/community college',
            'Undergraduate degree (BA/BSc/other)',
            'Graduate degree (MA/MSc/MPhil/other)',
            'Doctorate degree (PhD/other)',
            "Don't know / not applicable"
        ]
    },
    'p_depression': {
        'LongName': 'Prolific Depression',
        'Description': "Depression as repored in Prolific's screening question: Do you experience depression?",
        'Options': ['Yes', 'No', 'Prefer not to say']
    },
    'p_anxiety': {
        'LongName': 'Prolific Anxiety',
        'Description': "Anxiety as repored in Prolific's screening question: Do you experience anxiety?",
        'Options': ['Yes', 'No', 'Prefer not to say']
    },
    'p_attention': {
        'LongName': 'Prolific Attention',
        'Description': "Attention as repored in Prolific's screening question: Do you consider yourself to have attention deficit disorder (ADD)/attention deficit hyperactivity disorder (ADHD)?",
        'Options': ['Yes', 'No', 'Prefer not to say']
    },
    'p_mental_health_ongoing': {
        'LongName': 'Prolific Mental Health Ongoing',
        'Description': "Mental Health Ongoing as repored in Prolific's screening question: Do you have – or have you had – a diagnosed, on-going mental health/illness/condition?",
        'Options': ['Yes', 'No', 'Prefer not to say']
    },
    'p_mental_illness_impact': {
        'LongName': 'Prolific Mental Health Impact',
        'Description': "Mental Health Impact as repored in Prolific's screening question: Do you have any diagnosed mental health condition that is uncontrolled (by medication or intervention) and which has a significant impact on your daily life / activities?",
        'Options': ['Yes', 'No', 'Rather not to say']
    },
    'prolific_screen_group':{
        'LongName': 'Prolific Screening Group',
        'Description': 'Prolific recruitment was conducted in specific bins based on response to prolific scrreners. This field represents the bin the participant was recurited in. Since responses to screening questions may have differed from respones when I collected the same question, this differs from the screening group derived from respones to the CogMood survey. Calculated from p_depression, p_anxiety, p_attention, and p_mental_health_ongoing.'
    }
}

In [18]:
for kk in data_dict.keys():
    if kk[:5] == 'today':
        if data_dict[kk]['LongName'] == '':
            print(kk)
            try:
                data_dict[kk]['LongName'] = 'Today ' + data_dict[kk[5:]]['LongName']
                data_dict[kk]['Description'] = data_dict[kk[5:]]['Description']
                data_dict[kk]['Stem'] = 'Please select the statements that apply to you **today**'
            except KeyError:
                data_dict[kk]['LongName'] = 'Today ' + data_dict[kk[5:-1] + '_' + kk[-1]]['LongName']
                data_dict[kk]['Description'] = data_dict[kk[5:-1] + '_' + kk[-1]]['Description']
                data_dict[kk]['Stem'] = 'Please select the statements that apply to you **today**'


todaybaars_inattentive_1
todaybaars_inattentive_2
todaybaars_inattentive_3
todaybaars_inattentive_4
todaybaars_inattentive_5
todaybaars_inattentive_6
todaybaars_inattentive_7
todaybaars_inattentive_8
todaybaars_inattentive_9
todaybaars_hyperactivity_1
todaybaars_hyperactivity_2
todaybaars_hyperactivity_3
todaybaars_hyperactivity_4
todaybaars_hyperactivity_5
todaybaars_impulsivity_1
todaybaars_impulsivity_2
todaybaars_impulsivity_3
todaybaars_impulsivity_4
todaybaars_sct_1
todaybaars_sct_2
todaybaars_sct_3
todaybaars_sct_4
todaybaars_sct_5
todaybaars_sct_6
todaybaars_sct_7
todaybaars_sct_8
todaybaars_sct_9
todaygad7_1
todaygad7_2
todaygad7_3
todaygad7_4
todaygad7_5
todaygad7_6
todaygad7_7
todayphq8_1
todayphq8_2
todayphq8_3
todayphq8_4
todayphq8_5
todayphq8_6
todayphq8_7
todayphq8_8
todayhitop_anhdep_1
todayhitop_sepinsec_1
todayhitop_anxwor_1
todayhitop_welbe_1
todayhitop_appgn_1
todayhitop_anhdep_2
todayhitop_sepinsec_2
todayhitop_anxwor_2
todayhitop_sepinsec_3
todayhitop_socanx_1
tod

In [19]:
for kk in data_dict.keys():
    data_dict[kk]['dtype'] = str(psrdf[kk].dtype)
    data_dict[kk]['min'] = psrdf[kk].min()
    data_dict[kk]['max'] = psrdf[kk].max()


In [20]:
for kk in psrdf.columns:
    if kk not in data_dict.keys():
        print(f'{kk} not in data_dict')

survey_date not in data_dict


In [21]:
(survey_dir / 'data_dict.json').write_text(json.dumps(data_dict, indent=2))

144998

In [25]:
psrdf.group_by('prolific_screen_group').len()

prolific_screen_group,len
str,u32
"""dep""",155
"""hv""",806
"""dep_atn""",39
"""anx_atn""",243
null,545
"""dep_anx""",785
"""dep_anx_atn""",758
"""anx""",682
"""atn""",55
